## Set up
Importante east us: ada

OpenAI Version: 0.27.4

In [34]:
# pip install matplotlib
# pip install plotly
# pip install scipy
# pip install scikit-learn


Importante east us

In [35]:
import openai
import os
import pandas as pd
import numpy as np
import sys
import tiktoken

from dotenv import load_dotenv
from openai.embeddings_utils import get_embedding, cosine_similarity

In [36]:
load_dotenv()

openai.api_type: str = "azure"
openai.api_key = os.getenv("AZURE_OAI_KEY_EMBEDDING")
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT_EMBEDDING")
openai.api_version = os.getenv("OPENAI_API_VERSION_EMBEDDING")

In [37]:
input_datapath = "recipes_onecol.csv"

df = pd.read_csv(input_datapath)
df.head()

,Recipe
0,Recipe: Caramelized Bread Pudding with Chocola...
1,Recipe: White Sauce or Bechamel Sauce - Ingred...
2,"Recipe: Grilled Whole Mackerel with Lemon, Ore..."
3,Recipe: Christmas Croquembouche - Ingredients:...
4,Recipe: Chocolate Roll-Out Cookies - Ingredien...


## Utilizando un modelo Ada para soportar embeddings

In [38]:
embedding_encoding = "cl100k_base"
# the maximum for text-embedding-ada-002 is 8191
max_tokens = 8000
# the number of reviews to embed
top_n = 1000
encoding = tiktoken.get_encoding(embedding_encoding)

tokens por cada fila

In [39]:
df["n_tokens"] = df.Recipe.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
df

,Recipe,n_tokens
0,Recipe: Caramelized Bread Pudding with Chocola...,746
1,Recipe: White Sauce or Bechamel Sauce - Ingred...,396
2,"Recipe: Grilled Whole Mackerel with Lemon, Ore...",749
3,Recipe: Christmas Croquembouche - Ingredients:...,1210
4,Recipe: Chocolate Roll-Out Cookies - Ingredien...,1047
...,...,...
494,Recipe: Mozzarella Arrabiata Salsa - Ingredien...,236
495,Recipe: Tangy Frozen Greek Yogurt - Ingredient...,245
496,Recipe: Belgian Ale-Braised Brisket - Ingredie...,739
497,Recipe: Banana and Mango Spring Rolls with Coc...,813


Crear embeddings

In [40]:
from openai.embeddings_utils import get_embedding, cosine_similarity
def openai_text_embeddings(text):
    """
    Generating embeddings from text using Azure Open AI
    """
    embeddings = get_embedding(
      text,
        engine="text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )

    return embeddings

In [41]:
emb = openai_text_embeddings("Mi nombre es Maritza Mera")
len (emb)


1536

In [43]:

#si obtiene un error por llegar a límites de cuota, eliminé unas filas del dataset con la siguiente línea de código y vuelva a ejecutar el cálculo de embeddings
#df.drop(df.index[100:600],  inplace=True)

df["embedding"] = df.Recipe.apply(lambda x: openai_text_embeddings(x))


### Aplicar embeddings a todos los registros del dataframe

In [44]:

df

,Recipe,n_tokens,embedding
0,Recipe: Caramelized Bread Pudding with Chocola...,746,"[0.011631493456661701, -0.004553870297968388, ..."
1,Recipe: White Sauce or Bechamel Sauce - Ingred...,396,"[0.010483422316610813, 0.001243288628757, 0.00..."
2,"Recipe: Grilled Whole Mackerel with Lemon, Ore...",749,"[0.035249724984169006, 0.015324844047427177, -..."
3,Recipe: Christmas Croquembouche - Ingredients:...,1210,"[-0.011104307137429714, -0.0020673261024057865..."
4,Recipe: Chocolate Roll-Out Cookies - Ingredien...,1047,"[-0.013839004561305046, -0.0015087819192558527..."
...,...,...,...
95,Recipe: Creamed Pearl Onions and Fennel with C...,688,"[0.01678382232785225, 0.0024797325022518635, 0..."
96,Recipe: 3-Ingredient Morning Buns - Ingredient...,332,"[-0.0036542131565511227, -0.00764241348952055,..."
97,Recipe: Rosemary-Concord Shrub - Ingredients: ...,310,"[0.033475566655397415, 0.002127485116943717, 0..."
98,Recipe: Charred Caprese Sandwich - Ingredients...,685,"[0.0004337150603532791, -0.011011882685124874,..."


In [45]:

#Ejecute estas líneas de código para leer el dataset con los embeddings en un archivo csv en caso que no haya podido hacer cálculo de embeddings por límites de cuota
datafile_path = "recipes_onecol_with_embeddings.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(eval).apply(np.array)
df

,Unnamed: 0,Recipe,n_tokens,embedding
0,0,Recipe: Caramelized Bread Pudding with Chocola...,746,"[0.009514863602817059, -0.004959321580827236, ..."
1,1,Recipe: White Sauce or Bechamel Sauce - Ingred...,396,"[0.006938737351447344, 5.168988718651235e-05, ..."
2,2,"Recipe: Grilled Whole Mackerel with Lemon, Ore...",749,"[0.0321684330701828, 0.015827074646949768, -0...."
3,3,Recipe: Christmas Croquembouche - Ingredients:...,1210,"[-0.013321062549948692, -0.003860416589304805,..."
4,4,Recipe: Chocolate Roll-Out Cookies - Ingredien...,1047,"[-0.015186915174126625, -0.0015019221464172006..."
...,...,...,...,...
494,494,Recipe: Mozzarella Arrabiata Salsa - Ingredien...,236,"[0.007525882683694363, 0.0012005887692794204, ..."
495,495,Recipe: Tangy Frozen Greek Yogurt - Ingredient...,245,"[0.008731059730052948, -0.006505751982331276, ..."
496,496,Recipe: Belgian Ale-Braised Brisket - Ingredie...,739,"[0.002623694483190775, -0.0018848098115995526,..."
497,497,Recipe: Banana and Mango Spring Rolls with Coc...,813,"[-0.005268465261906385, -0.009891543537378311,..."


## Función para hacer una pregunta y obtener una respuesta evaluando similiradidad

In [46]:
def ask_question(question, n=1):
    question_embedding = openai_text_embeddings(
        question,
    )

    df["similarity"] = df.embedding.apply(
        lambda x: cosine_similarity(x, question_embedding)
    )

    results = df.sort_values("similarity", ascending=False).head(n)
    answer = " ".join(results.Recipe.tolist())
    print("Answer:", answer)

In [47]:
ask_question("Chocolate and berries")


Answer: Recipe: Raspberry Sundaes with Chocolate Sauce and Roasted Almonds - Ingredients: 6 ounces bittersweet chocolate, finely chopped;3/4 cup water;2/3 cups sugar;2 tablespoons honey;3 tablespoons unsalted butter;Pinch of kosher salt;1 pint fresh raspberries (12 ounces);2 tablespoons sugar;2 tablespoons elderflower liqueur, such as St-Germain;2 tablespoons fresh lemon juice;2 pints vanilla ice cream;Chopped roasted unsalted almonds, for garnish- Instructions: To make the chocolate sauce, place a saucepan over medium-low heat, combine the chocolate, and water, vigorously stirring until completely melted and smooth. Let it come to a low simmer until small bubbles appear around the outside. Add sugar, honey and keep at a low simmer until thickened, about 3 minutes. Remove from heat and stir in butter and salt. Chocolate sauce will smooth out at is cools to room temperature. Store in an airtight container. In a bowl, gently toss together the raspberries, sugar, elderflower liqueur, and 

In [48]:
ask_question("Fish no alcohol")

Answer: Recipe: Soft Fish Tacos - Ingredients: 1/4 cup mayonnaise;1/4 cup ketchup;1/4 cup crema mexicana*;1 cup all purpose flour;1 teaspoon fine sea salt;1/2 teaspoon ground pepper;1 cup dark beer, room temperature;13/4 pounds halibut, cut into 5x3/4-inch strips;1 lime, halved crosswise;12 white corn tortillas;Vegetable oil (for deep-frying);1 1/2 cups shredded red cabbage;2 large tomatoes, chopped;Lime wedges;Bottled hot pepper sauce- Instructions: For sauce: Mix all ingredients in bowl; season with salt and pepper.
Mix all ingredients in bowl; season with salt and pepper.
For batter and fish: Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes. Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter. Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 minute. Sprinkle top with wate